In [1]:
import json
import pandas as pd
import urllib.request
from pprint import pprint
import datetime
import time

In [34]:
CMC_data_last = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
CMC_data_last = CMC_data_last[["rank", "name", "price_usd", "price_btc", "24h_volume_usd", "percent_change_1h", "percent_change_24h"]]
CMC_data_last["time"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
CMC_data_last_price_only = CMC_data_last[["rank", "name", "price_usd", "time"]]
CMC_data_30_ago = CMC_data_last_price_only
CMC_data_25_ago = CMC_data_last_price_only
CMC_data_20_ago = CMC_data_last_price_only
CMC_data_15_ago = CMC_data_last_price_only
CMC_data_10_ago = CMC_data_last_price_only
CMC_data_05_ago = CMC_data_last_price_only

In [37]:
starttime=time.time()
while True:
    CMC_data_30_ago = CMC_data_25_ago
    CMC_data_25_ago = CMC_data_20_ago
    CMC_data_20_ago = CMC_data_15_ago
    CMC_data_15_ago = CMC_data_10_ago
    CMC_data_10_ago = CMC_data_05_ago
    CMC_data_05_ago = CMC_data_last_price_only
    CMC_data_last = pd.read_json("https://api.coinmarketcap.com/v1/ticker")
    CMC_data_last = CMC_data_last[["rank", "name", "price_usd", "price_btc", "24h_volume_usd", "percent_change_1h", "percent_change_24h"]]
    CMC_data_last["time"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    CMC_data_last_price_only = CMC_data_last[["rank", "name", "price_usd", "time"]]
    CMC_data_delta = CMC_data_last_price_only.rename(columns = {'price_usd':'price_00'}).merge(CMC_data_05_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_05'}), on="name").merge(CMC_data_10_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_10'}), on="name").merge(CMC_data_20_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_20'}), on="name").merge(CMC_data_30_ago[["name", "price_usd"]].rename(columns = {'price_usd':'price_30'}), on="name")
    CMC_data_delta["percent_change_05m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_05"])/(CMC_data_delta["price_05"]),5)*100
    CMC_data_delta["percent_change_10m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_10"])/(CMC_data_delta["price_10"]),5)*100
    CMC_data_delta["percent_change_20m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_20"])/(CMC_data_delta["price_20"]),5)*100
    CMC_data_delta["percent_change_30m"] = round((CMC_data_delta["price_00"] - CMC_data_delta["price_30"])/(CMC_data_delta["price_30"]),5)*100
    CMC_data_last = CMC_data_last.merge(CMC_data_delta[["name", "percent_change_05m", "percent_change_10m", "percent_change_20m", "percent_change_30m"]], on="name")

    time.sleep(10.0 - ((time.time() - starttime) % 10.0))

KeyboardInterrupt: 

In [ ]:
CMC_data_last[CMC_data_last["24h_volume_usd"]>1000000].sort_values(by = "percent_change_05m", ascending=False)